In [1]:
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
import openpyxl
import customtkinter as ctk
from customtkinter import filedialog
import funciones
from funciones import *


### Subir el archivo y guardarlo en un dataframe

In [2]:
#cargar primero el padron de sidefi
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padron = pd.read_excel(ruta_archivo, sheet_name=1)
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")



<_io.TextIOWrapper name='G:/Mi unidad/osoetsylra/importaciones_TIS_SD_ETC_a_sidefi/TIS/Padron completo tintoreros 11-23 Actual.xlsx' mode='r' encoding='cp1252'>
     CUIL  TITULAR  CUIL FAMILIAR          APELLIDO             NOMBRE  \
0      23291190944            NaN          MERCANTI     ANDREA MARIANA   
1      23291190944   2.753850e+10  ROBERTO MERCANTI            AYMARA    
2      23291190944   2.744871e+10          ROBERTO       MAYRA MICAELA   
3      27407483834            NaN          MARTINEZ            LUDMILA   
4      27407483834   2.753529e+10          MARTINEZ        IARA ELUNEY   
..             ...            ...               ...                ...   
277    23306777254   2.046945e+10            RANDO     TIZIANO NAHUEL    
278    23306777254   2.356413e+10            RANDO      KENDRA YAZMIN    
279    23306777254   2.049538e+10            RANDO   LAUTARO EZEQUIEL    
280    27391446348            NaN          MAYORGA       NADIA GISELLA   
281    27381431210       

#### Calcular edades

In [4]:
def find_error(date_str):
    if not isinstance(date_str, str):
        date_str = str(date_str)
    if '"' in date_str:
        return date_str.find('"')
    else:
        return -1  # Devuelve -1 si no se encuentra el caracter problemático
    
padron['error_position'] = padron['FECHA DE NAC.*'].apply(find_error)
registros_con_error = padron[padron['error_position'] != -1]

file_path = filedialog.asksaveasfilename(defaultextension='.csv')

# Si se seleccionó un archivo, guarda el dataframe en la ruta seleccionada
if file_path:
    if file_path.endswith('.csv'):
        registros_con_error.to_csv(file_path, sep = ";", encoding = "ISO-8859-1",index=False)
    elif file_path.endswith('.xlsx'):
        writer = pd.ExcelWriter(file_path, engine='xlsxwriter')
        registros_con_error.to_excel(writer, sheet_name='Hoja1', index=False)
        writer.save()
    else:
        print('Extensión de archivo no válida.')


0      1991-10-25 00:00:00
1      2014-05-13 00:00:00
2      2003-04-25 00:00:00
3      1997-10-24 00:00:00
4      2013-11-14 00:00:00
              ...         
277    2005-11-07 00:00:00
278    2017-08-01 00:00:00
279    2009-05-06 00:00:00
280    1990-03-25 00:00:00
281    1995-03-27 00:00:00
Name: FECHA DE NAC.*, Length: 282, dtype: object


### corregir fechas + calcular edades


In [ ]:
#fecha de nacimiento calcular edad
def calcular_edad(fecha_nacimiento):
    if pd.isnull(fecha_nacimiento):
        return None
    try:
        fecha_nacimiento = datetime.strptime(str(fecha_nacimiento), "%d/%m/%Y")
        edad = int((datetime.now() - fecha_nacimiento).days / 365.25)
        return edad
    except ValueError:
        return "Error: formato de fecha inválido"

# crear campo edad
padron['edad'] = padron['Fecha Nacimiento'].apply(calcular_edad)
edades = padron[['edad','Fecha Nacimiento','Apellido Afiliado','Nombre Afiliado','Nro. Documento']]

file_path = filedialog.asksaveasfilename(defaultextension='.csv')

# Si se seleccionó un archivo, guarda el dataframe en la ruta seleccionada
if file_path:
    if file_path.endswith('.csv'):
        edades.to_csv(file_path, sep = ";", encoding = "cp1252",index=False)
    elif file_path.endswith('.xlsx'):
        writer = pd.ExcelWriter(file_path, engine='xlsxwriter')
        edades.to_excel(writer, sheet_name='Hoja1', index=False)
        writer.save()
    else:
        print('Extensión de archivo no válida.')

### Diccionarios

In [38]:
sedes_localidades={
"Berisso":"LA PLATA",
"Moreno":"SAN JUSTO",
"Rafael Castillo":"SAN JUSTO",
"Lomas De Zamora":"QUILMES",
"Gregorio De Laferrere":"SAN JUSTO",
"Yerba Buena":"TUCUMAN",
"Pilar":"SAN MARTIN",
"Virrey Del Pino":"SAN JUSTO",
"Gregorio De La ferrere":"SAN JUSTO",
"San Miguel":"SAN MARTIN",
"Villa Luzuriaga":"SAN JUSTO",
"Alejandro Korn":"QUILMES",
"Gonzales Catan":"SAN JUSTO",
"Jose C Paz":"SAN MARTIN",
"Gonzalez Catan":"SAN JUSTO",
"Isidro Casanova":"SAN JUSTO",
"San Jose":"QUILMES",
"Boulogne":"SAN MARTIN",
"Moron":"SAN JUSTO",
"L Del Mirador":"SAN JUSTO",
"Mar Del Plata":"MAR DEL PLATA",
"Ezeiza":"QUILMES",
"San Miguel De Tucuman":"TUCUMAN",
"Quilmes":"QUILMES",
"Jose Marmol":"ZONA SUR",
"S. S. De Jujuy":"CORDOBA",
"Tandil":"MAR DEL PLATA",
"Tigre":"SAN MARTIN",
"Pablo Podesta":"SAN JUSTO",
"La Plata":"LA PLATA",
"Lanus Este":"QUILMES",
"Escobar":"SAN MARTIN",
"Florencio Varela":"QUILMES",
"Villa Rosa":"SAN MARTIN",
"Merlo":"SAN JUSTO",
"Caba":"CHILE",
"Villa Madero":"SAN JUSTO",
"Bernal Oeste":"QUILMES",
"El Palomar":"SAN JUSTO",
"Almirante Brown":"QUILMES",
"Villa Celina":"SAN JUSTO",
"Lomas Del Mirador":"SAN JUSTO",
"Brandsen":"LA PLATA",
"Rafael Calzada":"QUILMES",
"Beccar":"SAN MARTIN",
"Lastenia":"TUCUMAN",
"General Rodriguez":"SAN JUSTO",
"Hurlingham":"SAN MARTIN",
"Pontevedra":"SAN JUSTO",
"Glew":"QUILMES",
"Esteban Echeverria":"QUILMES",
"San Justo":"SAN JUSTO",
"Canning":"QUILMES",
"Don Torcuato":"SAN MARTIN",
"Suarez":"SAN MARTIN",
"Loma Hermosa":"SAN MARTIN",
"Berazategui":"QUILMES",
"G Catan":"SAN JUSTO",
"Avellaneda":"QUILMES",
"Ciudad Evita":"SAN JUSTO",
"Colombres (cruz Alta)":"TUCUMAN",
"San Fernando":"SAN MARTIN",
"Tafi Viejo":"TUCUMAN",
"Banda Rio Sali":"TUCUMAN",
"Burzaco":"QUILMES",
"Banfield":"QUILMES",
"Ciudadela":"SAN JUSTO",
"Trujui":"SAN JUSTO",
"La Union":"QUILMES",
"Castelar":"SAN JUSTO",
"Jose C  Paz":"SAN MARTIN",
"Cruz Alta":"TUCUMAN",
"Monte Grande":"QUILMES",
"Ramos Mejia":"SAN JUSTO",
"L.Mirador":"SAN JUSTO",
"San Francisco Solano":"QUILMES",
"Claypole":"QUILMES",
"Marcos Paz":"SAN JUSTO",
"El Jaguel":"QUILMES",
"General San Martin":"SAN MARTIN",
"La Tablada":"SAN JUSTO ",
"Lanus":"QUILMES",
"Puerto Madryn":"MENDOZA",
"Los Pocitos":"CHILE",
"Vicente Lopez":"SAN MARTIN",
"Ituzaingo":"SAN JUSTO",
"Alderetes":"TUCUMAN",
"Cevil Redondo":"TUCUMAN",
"Ensenada":"LA PLATA",
"Villa Bosch":"SAN MARTIN",
"Caseros":"SAN MARTIN",
"Balcarce":"MAR DEL PLATA",
"Villa De Mayo":"SAN MARTIN",
"Ranchillos":"TUCUMAN",
"San Matin":"SAN MARTIN",
"La Quiaca":"CHILE",
"Parque San Martin":"SAN JUSTO",
"San Nicolas":"ROSARIO ",
"V. Scasso":"SAN JUSTO",
"Libertad":"SAN JUSTO",
"Los Polvorines":"SAN MARTIN",
"Villa Dominico":"QUILMES",
"Nicole":"SAN JUSTO",
"Bella Vista":"SAN MARTIN",
"Domselaar":"QUILMES",
"Wilde":"QUILMES",
"Mataderos":"SAN JUSTO",
"Villa Carmela":"TUCUMAN",
"Dock Sud":"QUILMES",
"Villa Lugano":"CHILE",
"Lugano":"CHILE",
"San Antonio De Padua":"SAN JUSTO",
"Guernica":"QUILMES",
"Cañuelas":"SAN MARTIN",
"Lanus Oeste":"QUILMES",
"Manuel Alberti":"SAN MARTIN",
"Jose Leon Suarez":"SAN MARTIN",
"Mariano Acosta":"SAN JUSTO",
"Adolfo Sourdeaux":"SAN MARTIN",
"Longchamps":"QUILMES",
"Gerli":"QUILMES",
"V. Del Pino":"SAN JUSTO",
"9 De Abril":"QUILMES",
"Presidente Derqui":"SAN MARTIN",
"Vila Rosa":"SAN MARTIN",
"Buenos Aires":"QUILMES",
"La Matanza":"SAN JUSTO",
"L. Mirador":"SAN JUSTO",
"Gregorio La Ferrere":"SAN JUSTO",
"Virreydelpino":"SAN JUSTO",
"Neuquen":"MENDOZA",
"Villa Regina/ Rio Negro":"MENDOZA",
"Merlo Norte":"SAN JUSTO",
"Ingeniero Pablo Nogues":"SAN MARTIN",
"Merlos Gomez":"SAN JUSTO",
"Villa Tesei":"SAN JUSTO",
"El Libertador":"SAN MARTIN",
"Pablo Nogues":"SAN MARTIN",
"Tortuguitas":"SAN MARTIN",
"Virrel Del Pino":"SAN JUSTO",
"Remedios De Escalada":"QUILMES",
"Ingeniero Adolfo Sordeaux":"SAN MARTIN",
"Grand Bourg":"SAN MARTIN",
"Ingeniero Budge":"QUILMES",
"Bernal Este":"QUILMES",
"Villa Adelina":"SAN MARTIN",
"Barrio Santo Tomas":"SAN MARTIN",
"Zelaya":"SAN MARTIN",
"Benavidez":"SAN MARTIN",
" ":"CHILE",
"Bernal":"QUILMES",
"San Vicente":"QUILMES",
"V.Del Pino":"SAN JUSTO",
"V Del Pino":"SAN JUSTO",
"Malvinas Argentinas":"SAN MARTIN",
"Mercado Central":"SAN JUSTO",
"Puente Ezcurra":"SAN JUSTO",
"Barrio 17 De Marzo":"SAN JUSTO",
"Virrreyes":"SAN MARTIN",
"Jose Ingenieros":"SAN ",
"Manzanares":"SAN MARTIN",
"Cuidadela":"SAN JUSTO",
"Banda Del Rio Sali":"TUCUMAN",
"Rincon De Milberg":"SAN MARTIN",
"Martin Coronado":"SAN JUSTO",
"Luis Guillon":"QUILMES",
"William Morris":"SAN MARTIN",
"Ministro Rivadavia":"QUILMES",
"Villa Gonnet":"QUILMES",
"Villa Lujan":"TUCUMAN",
"Villa Hidalgo":"SAN MARTIN",
"Ranelagh":"QUILMES",
"Segurola 1964":"CHILE",
"Sarandi":"QUILMES",
"Sourdeaux":"SAN MARTIN",
"El Talar":"SAN MARTIN",
"Villa 31":"CHILE",
"Jose C. Paz":"SAN MARTIN",
"San Isidro":"SAN MARTIN",
"Misiones":"CHILE",
"Acasusso":"SAN MARTIN",
"Paso Del Rey":"SAN JUSTO",
"San Martin":"SAN MARTIN",
"La Boca":"CHILE",
"La Reja":"SAN JUSTO",
"Cruz Alta-tucuman":"TUCUMAN",
"Victoria":"SAN MARTIN",
"Delta Tigre":"SAN MARTIN",
"Virry Del Pino":"SAN JUSTO",
"V. Soldati":"CHILE",
"Floresta":"CHILE",
"Spegazinni":"QUILMES",
"Matanza":"SAN JUSTO",
"Loma De Zamora":"QUILMES",
"San fernando":"SAN MARTIN",
"9 De Abril":"LA PLATA",
"La Tablada":"SAN JUSTO",
"San fernando":"SAN MARTIN",
"Lomas del Mirador":"SAN JUSTO",
"Jose Ingenieros":"SAN MARTIN",
'Cevil Redodo':"TUCUMAN",
'Garín':"SAN MARTIN",
'Guernicaa':'QUILMES',
'Puente Escurra':"SAN JUSTO"

}

sede_codigo={
"LA PLATA":"8",
"SAN JUSTO":"11",
"QUILMES":"14",
"TUCUMAN":"13",
"SAN MARTIN":"12",
"MAR DEL PLATA":"6",
"ZONA SUR":"14",
"CORDOBA":"2",
"CHILE":"10",
"MENDOZA":"1",
"ROSARIO ":"5"
}

### NORMALIZAR LOCALIDADES

In [39]:
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([ "gregorio laferrere",
'Laferrere',
'G. De Laferrere',
'G. De La Ferrere',
'G. Laferrere',
'G.Laferrere',
'G Laferrere',
'G, Laferrere',
'G.De Lafererre',
'G.De Laferrere',
'G.Laferrer',"g. laferrere", "laferrere" ], "Gregorio De La ferrere" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'A.Korn',
'Alejandro Korh'
    ], "Alejandro Korn" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'C. Evita',
'C Evita'
    ], "Ciudad Evita" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'Castillo',
'R. Castillo',
'R.Castillo',
'R Castillo'
    ], "Rafael Castillo" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'G.Catan',
'G. Catan',
'Catan',
    ], "Gonzales Catan" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'F.Varela'
    ], "Florencio Varela" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'Gral Rodriguez',
'Rodriguez',
'Gral Rodrigue,',
    ], "General Rodriguez" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'Gral Rodriguez',
'Rodriguez',
'Gral Rodrigue,',
    ], "General Rodriguez" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'I.Casanova',
'I. Casanova',
    ], "Isidro Casanova" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'L Dek Mirador'
    ], "L Del Mirador" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'L.De Zamora'
    ], "Lomas De Zamora" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'La Quiaca (JUJUY)'
    ], "La Quiaca" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'S Justo'
    ], "San Justo" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'Los Pocitos (PATAGONES)'
    ], "Los Pocitos" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'S.F.Solano',
'Solano'
    ], "San Francisco Solano" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'San Fernnado'
    ], "San fernando" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'R Mejia'
    ], "Ramos Mejia" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'Tablada'
    ], "La Tablada" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'V Luzuriaga','V.Luzuriaga'
    ], "Villa Luzuriaga" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'V.Madero',
'V.Maderl',
'V Madero'
    ], "Villa Madero" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'Villa De Mayo (MALVINAS ARGENTINAS)',
    ], "Villa De Mayo" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'San Miguel De Tuc.',
    ], "San Miguel De Tucuman" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'V. Scasso (GONZALEZ CATAN)',
    ], "V. Scasso " )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
'V. Celina',
'V Celina'
    ], "Villa Celina" )
padron["LOCALIDAD*"] = padron["LOCALIDAD*"].replace([
"9 De  Abril"
    ], "9 De Abril" )




In [40]:
padron['Sede']=padron['LOCALIDAD*'].map(sedes_localidades)
#segundo pasaje

padron['Sede']=padron['Sede'].map(sede_codigo)

In [41]:
padron['FECHA DE ALTA*']=padron['FECHA DE ALTA*'].astype(str).apply(funciones.transformar_fecha2)
padron['FECHA DE NAC.*']=padron['FECHA DE NAC.*'].astype(str)
padron['FECHA DE NAC.*']=padron['FECHA DE NAC.*'].apply(funciones.transformar_fecha2)

In [42]:


padron.to_excel(r'E:\Mi unidad\osoetsylra\importaciones_TIS_SD_ETC_a_sidefi\EVP\junio final\junio_final_nombre_fix.xlsx', index=False)